In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\vecteezy_busy-street-in-evening-hong-kong_28840705 (1) (1).mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4']

In [9]:
def get_contour_detections(mask, thresh=400):

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)
    detections = []
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        area = w*h
        if area > thresh:
            detections.append([x,y,x+w,y+h, area])

    return np.array(detections)

def remove_contained_bboxes(boxes):
    check_array = np.array([True, True, False, False])
    keep = list(range(0, len(boxes)))
    for i in keep:
        for j in range(0, len(boxes)):
            if np.all((np.array(boxes[j]) >= np.array(boxes[i])) == check_array):
                try:
                    keep.remove(j)
                except ValueError:
                    continue
    return keep


def non_max_suppression(boxes, scores, threshold=1e-1):
    boxes = boxes[np.argsort(scores)[::-1]]

    order = remove_contained_bboxes(boxes)

    keep = []
    while order:
        i = order.pop(0)
        keep.append(i)
        for j in order:
            intersection = max(0, min(boxes[i][2], boxes[j][2]) - max(boxes[i][0], boxes[j][0])) * \
                           max(0, min(boxes[i][3], boxes[j][3]) - max(boxes[i][1], boxes[j][1]))
            union = (boxes[i][2] - boxes[i][0]) * (boxes[i][3] - boxes[i][1]) + \
                    (boxes[j][2] - boxes[j][0]) * (boxes[j][3] - boxes[j][1]) - intersection
            iou = intersection / union

            if iou > threshold:
                order.remove(j)
                
    return boxes[keep]

def draw_bboxes(frame, detections):
    for det in detections:
        x1,y1,x2,y2 = det
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)

In [10]:
def compute_flow(frame1, frame2):
    # convert to grayscale
    #gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    #gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    gray1 = cv2.GaussianBlur(frame1, dst=None, ksize=(3,3), sigmaX=5)
    gray2 = cv2.GaussianBlur(frame2, dst=None, ksize=(3,3), sigmaX=5)

    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None,
                                        pyr_scale=0.75,
                                        levels=3,
                                        winsize=5,
                                        iterations=3,
                                        poly_n=10,
                                        poly_sigma=1.2,
                                        flags=0)
    return flow

def get_flow_viz(flow):
    hsv = np.zeros((flow.shape[0], flow.shape[1], 3), dtype=np.uint8)
    hsv[..., 1] = 255

    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    return rgb

In [11]:
def get_motion_mask(flow_mag, motion_thresh=1, kernel=np.ones((7,7))):

    motion_mask = np.uint8(flow_mag > motion_thresh)*255

    motion_mask = cv2.erode(motion_mask, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    
    return motion_mask

In [12]:
cap = cv2.VideoCapture(video_files[4])
ret, frame = cap.read()

mask_kernel = np.ones((7,7), dtype=np.uint8)
height, width, layers = frame.shape
new_h = height / 2
new_w = width / 2
motion_thresh = np.c_[np.linspace(0.3, 1, int(new_h))].repeat(int(new_w), axis=-1)

first_frame = None

while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret == True:
      
        height, width, layers = frame.shape
        new_h = height / 2
        new_w = width / 2
        frame = cv2.resize(frame, (int(new_w), int(new_h)))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if first_frame is None:
            first_frame = gray
            continue

        flow = compute_flow(first_frame, gray)

        mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        motion_mask = get_motion_mask(mag, motion_thresh=motion_thresh, kernel=mask_kernel)

        detections = get_contour_detections(motion_mask, 400)

        bboxes = detections[:, :4]
        scores = detections[:, -1]

        nms_bboxes = non_max_suppression(bboxes, scores, threshold=0.1)

        for box in nms_bboxes:
            frame = cv2.rectangle(frame, (box[0],box[1]), (box[2],box[3]), (0,255,0), 2)


            
        cv2.imshow('normal video',frame)

    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        first_frame = gray

    else:
        break

cap.release()

cv2.destroyAllWindows()